Classification of the MNIST dataset with the logistic regression in Theano



In [ ]:
#imports
import numpy as np
from theano import *
import theano.tensor as T
import cPickle, gzip

Load the dataset, download it:

In [ ]:
!wget http://deeplearning.net/data/mnist/mnist.pkl.gz
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = cPickle.load(f)

print 'Shapes:'
print '\tTraining:   ', train_set[0].shape, train_set[1].shape
print '\tValidation: ', valid_set[0].shape, valid_set[1].shape
print '\tTest:       ', test_set[0].shape, test_set[1].shape

50 thousand training images, 10 thousand validation images, 10 thousand test images; each set comes with a set of labels 

In [ ]:
# Weight vector shape: from 784 pixels to 10 possible classifications
W_shape = (10, 784)
b_shape = 10

W = shared(np.random.random(W_shape) - 0.5, name="W") #shared 
b = shared(np.random.random(b_shape) - 0.5, name="b")

In [ ]:
# Makes an input matrix (which can be the training, validation 
#or test set)
x = T.dmatrix("x") # N x 784 - Theano, symbolic variable
labels = T.dmatrix("labels") # N x 10 (learn 10 classes)

In [ ]:
# Output of our logistic regression: softmax function (sigmoid)
output = T.nnet.softmax(x.dot(W.transpose()) + b)

In [ ]:
#The model predicts whichever class has the highest output 
#on its corresponding unit:
prediction = T.argmax(output, axis=1)

In [ ]:
# error function: binary cross-entropy (cost function)
cost = T.nnet.binary_crossentropy(output, labels).mean()

In [ ]:
def encode_labels(labels, max_index):
    """Encode the labels into binary vectors."""
    # Allocate the output labels, all zeros.
    encoded = np.zeros((labels.shape[0], max_index + 1))
    
    # Fill in the ones at the right indices.
    for i in xrange(labels.shape[0]):
        encoded[i, labels[i]] = 1
    return encoded

Compiling the Theano functions
Theano’s compiler applies many optimizations of varying complexity to these symbolic expressions. These optimizations include:

use of GPU for computations
constant folding
merging of similar subgraphs, to avoid redundant calculation
arithmetic simplification (e.g. x*y/x -> y, --x -> x)
inserting efficient BLAS operations (e.g. GEMM) in a variety of contexts
using memory aliasing to avoid calculation and others...

In [ ]:
compute_prediction = function([x], prediction)
compute_cost = function([x, labels], cost)

grad_W = grad(cost, W)
grad_b = grad(cost, b)
# Compute the gradient of our error function

 We can now train it on our training set until it seems to converge
 using a heuristic adaptive step size (gradient descent)

In [ ]:
# Set up the updates we want to do
alpha = T.dscalar("alpha")
updates = [(W, W - alpha * grad_W),
           (b, b - alpha * grad_b)]

# Make our function. Have it return the cost!
train = function([x, labels, alpha],
                 cost,
                 updates=updates)

# Calculating cost 
labeled = encode_labels(train_set[1], 9)
alpha = 8.0
costs = []

In [ ]:
while True:
    costs.append(float(train(train_set[0], labeled, alpha))) #one iteration of gradient descent
    
    #print every 10 iterations
    if len(costs) % 10 == 0:
        print 'Epoch', len(costs), 'with cost', costs[-1], 'and alpha', alpha

    #change the value of alpha 
    if len(costs) > 2 and costs[-2] - costs[-1] < 0.0001:
        if alpha < 0.2:
            break
        else:
            alpha = alpha / 1.5


In [ ]:
#Let's make our prediction on the test set:

prediction = compute_prediction(test_set[0])

#calculate accuracy: compare predicted with actual label
def accuracy(predicted, actual):
    total = 0.0
    correct = 0.0
    for p, a in zip(predicted, actual):
        total += 1
        if p == a:
            correct += 1
    return correct / total

accuracy(prediction, test_set[1])

print accuracy(prediction, test_set[1])